In [1]:
import glob
import os
import numpy as np

## Boilerplate path hack to give access to full clustered_SNe package
import sys
if __package__ is None:
    if os.pardir not in sys.path[0]:
        file_dir = os.getcwd()
        sys.path.insert(0, os.path.join(file_dir, 
                                        os.pardir, 
                                        os.pardir))
from clustered_SNe.analysis.parse import Overview, parse_run


/Users/egentry/anaconda/lib/python3.4/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [2]:
data_dir = "../saved_runs/cluster_parameter_study_new/"
data_dir_high_density = "../saved_runs/sample_mass_scaling_high_density/"
# data_dir = "../saved_runs/sample_mass_scaling_high_density"
data_dir_tmp = "../src/tmp/"

src_dir = "../src"
scripts_dir = "../scripts/"


In [3]:
def create_new_batch_file(id, destination_dir=scripts_dir, verbose=False):
    with open(os.path.join(scripts_dir, "restart.batch")) as f:
        text = f.read()
        with open(os.path.join(destination_dir, "restart.batch."+id), mode="w") as f2:
            text2 = text[:-1] + id
            if verbose:
                print(text2)
            f2.write(text2)
        
def checkpoint_99_exists(id, data_dir=data_dir):
    checkpoints = glob.glob(os.path.join(data_dir, id + "*_checkpoint_*.dat"))
    checkpoints = sorted(checkpoints)
    last_checkpoint = checkpoints[-1]
    last_checkpoint_num = int(last_checkpoint.split("_")[-1].strip(".dat")) % 100
    if last_checkpoint_num != 99:
        return False
    else:
        return True

def is_momentum_converged(id, data_dir=data_dir):
    _last_run = parse_run(data_dir, id)
    momenta = _last_run["momentum"]
    if (momenta.size < 25):
        earlier_momentum = momenta[0]
    else:
        earlier_momentum = momenta[-25]
    current_momentum = momenta[-1]
    tolerance = .01
    if (current_momentum > ((1+tolerance) * earlier_momentum)):
        return False
    else:
        return True
    


In [4]:
is_momentum_converged("ae1f1577-ad61-4f26-99ef-1204be73b417", data_dir=data_dir)

False

In [5]:
is_momentum_converged("a7bc1f23-aff9-4dcf-9705-03ae1f8e2507", data_dir=data_dir_high_density)

True

In [6]:
checkpoint_99_exists("ae1f1577-ad61-4f26-99ef-1204be73b417", data_dir = data_dir)

True

In [ ]:
overviews = glob.glob(os.path.join(data_dir, "*overview.dat"))
ids = []
for overview in overviews:
    id = os.path.basename(overview.split("_overview.dat")[0])
    ids.append(id)

In [ ]:
from multiprocessing import Pool
pool = Pool(4)
ids_tmp = np.array(ids)
finished = pool.map(checkpoint_99_exists, ids_tmp)
finished = np.array(finished)
converged = pool.map(is_momentum_converged, ids_tmp[finished])
converged = np.array(converged)

restart_ids = ids_tmp[finished][~converged]

In [ ]:
np.sum(finished[converged])

In [24]:
restart_ids

array(['0df337f0-6afa-496d-8eea-8e75f6a0bea5',
       '16f7d335-77b9-4ad9-8eba-153c57026ed8',
       '179c5f0b-ce52-4f81-84b2-b6afb47cb335',
       '34ad9744-fd58-43f8-ab50-09b499cf7115',
       '34ba2aec-f27c-4e09-9ffc-8d5e5fc6221d',
       '35991ff8-2185-4b73-bc31-5ab852e07387',
       '39758925-5bcf-475e-8f44-1e46e44cde75',
       '39fdd9f6-3d02-4d60-a3d1-2f3b392de77c',
       '3b381194-fe9b-4f6b-afc6-33e641f698c1',
       '40506e13-5e5e-408c-92cb-3f5b092eee62',
       '43b99111-b494-4a36-a2ab-486f0589c0b7',
       '45e9c742-7d2c-461b-ad63-5e1be00777c9',
       '487b841d-4822-482e-9276-a6cdcc541bb0',
       '4ab2cbde-90cd-4109-8754-6ef7f5a79898',
       '53c43e6e-1b48-4f96-bd93-1bf18ad5e25c',
       '55ad8e4c-2609-494f-939c-bf8ff137c148',
       '5608852a-c603-45c8-b363-d5fb6c1dbbb2',
       '572d20e4-89e5-48b9-983b-86f8833c2570',
       '57efb478-2ec7-4ace-9eea-71634d75b37a',
       '5bd8c70c-1f40-46fe-ae6a-1c25f63e8b92',
       '5d870455-d66c-437f-98ef-132f058b630c',
       '5f3b6

In [25]:
restart_ids.size

95

In [92]:
for restart_id in restart_ids:
    create_new_batch_file(restart_id)